In [1]:
import pandas as pd

In [10]:
df = pd.read_csv("Security_Updates_&_Hardware_Status.csv")
df = df.drop(columns=['Plateforme'])
df.head(5)

,Computer ID,System Type,RAM Usage (%),CPU Usage (%),Disk Usage (%),Network Status,MissingPatchsKB,affected microsoft product,Type of security risk,Severity,Download,Download (Link),CVE identifier(s)
0,CMP-0001,Workstation,50.9,35.6,60.8,Online,Release Notes,Microsoft Edge (Chromium-based),Unknown,Unknown,Security Update,Unknown,CVE-2025-8011
1,CMP-0002,Workstation,54.6,68.2,77.9,Online,Release Notes,Microsoft Edge (Chromium-based),Unknown,Unknown,Security Update,Unknown,CVE-2025-8010
2,CMP-0003,Workstation,55.5,66.1,72.8,Online,5002768,Microsoft SharePoint Server Subscription Edition,Spoofing,Important,Security Update,https://www.microsoft.com/en-us/download/detai...,CVE-2025-53771
3,CMP-0004,Workstation,63.5,36.3,51.6,Unstable,5002754,Microsoft SharePoint Server 2019,Spoofing,Important,Security Update,https://www.microsoft.com/en-us/download/detai...,CVE-2025-53771
4,CMP-0005,Workstation,31.4,42.7,82.4,Unstable,5002753,Microsoft SharePoint Server 2019,Spoofing,Important,Security Update,https://www.microsoft.com/en-us/download/detai...,CVE-2025-53771


In [14]:
import pandas as pd
import numpy as np
from collections import Counter
import datetime

# Load your dataset
df = pd.read_csv("Security_Updates_&_Hardware_Status.csv")

# Function to detect problems, assign score, and severity level
def detect_problems_and_score(df):
    problems_list = []
    critical_scores = []
    severity_levels = []

    for _, row in df.iterrows():
        issues = []
        score = 0

        # CPU Usage
        if row['CPU Usage (%)'] > 85:
            issues.append("High CPU usage")
            score += 2

        # RAM Usage
        if row['RAM Usage (%)'] > 80:
            issues.append("High RAM usage")
            score += 1.5

        # Disk Usage
        if row['Disk Usage (%)'] > 90:
            issues.append("Disk almost full")
            score += 2

        # Network
        if str(row['Network Status']).lower() != "connected":
            issues.append("Network disconnected")
            score += 3

        # Missing Patch
        if pd.notna(row['MissingPatchsKB']):
            issues.append("Missing patch")
            score += 2

        # Severity from dataset
        severity = str(row['Severity']).lower()
        if "critical" in severity:
            issues.append("Critical vulnerability")
            score += 3
        elif "important" in severity:
            issues.append("Important vulnerability")
            score += 2
        elif "moderate" in severity or "low" in severity:
            issues.append("Minor vulnerability")
            score += 1

        # Define severity level
        if score >= 7:
            level = "Critical"
        elif score >= 5:
            level = "High"
        elif score >= 3:
            level = "Medium"
        else:
            level = "Low"

        # Store results
        problems_list.append("; ".join(issues))
        critical_scores.append(score)
        severity_levels.append(level)

    # Add new columns to the dataset
    df['Problems'] = problems_list
    df['Critical_Score'] = critical_scores
    df['Severity_Level'] = severity_levels
    df['Is_Critical'] = df['Critical_Score'] >= 5

    return df

# Apply the function
df = detect_problems_and_score(df)

# Optional: Save the updated dataset
output_path= "critical_updated_dataset.csv"
df.to_csv(output_path, index=False)

# Create subset of only critical machines
df_critical = df[df['Severity_Level'] == "Critical"]
critical_output_path = "only_critical_machines.csv"
df_critical.to_csv(critical_output_path, index=False)

# Count of severity levels
severity_counts = df['Severity_Level'].value_counts()

# Most frequent problems
all_issues = ";".join(df['Problems'].dropna()).split(";")
issue_counts = Counter([i.strip() for i in all_issues])
top_issues = issue_counts.most_common(10)

# Top 10 most critical machines
top_critical_machines = df.sort_values(by='Critical_Score', ascending=False).head(10)

(output_path, critical_output_path, severity_counts, top_issues, top_critical_machines[['Computer ID', 'Critical_Score', 'Severity_Level']])




('critical_updated_dataset.csv',
 'only_critical_machines.csv',
 Severity_Level
 Critical    4782
 High         121
 Name: count, dtype: int64,
 [('Network disconnected', 4903),
  ('Missing patch', 4903),
  ('Important vulnerability', 4373),
  ('Critical vulnerability', 409),
  ('Minor vulnerability', 65)],
      Computer ID  Critical_Score Severity_Level
 4871    CMP-4872               8       Critical
 4845    CMP-4846               8       Critical
 10      CMP-0011               8       Critical
 4900    CMP-4901               8       Critical
 519     CMP-0520               8       Critical
 773     CMP-0774               8       Critical
 4859    CMP-4860               8       Critical
 4860    CMP-4861               8       Critical
 4861    CMP-4862               8       Critical
 4862    CMP-4863               8       Critical)